In [2]:
import numpy as np
import pandas as pd
import nibabel as nib

Load a pre-made 4D image containing binary lesion masks from multiple patients.

In [3]:
mask_stack_path = '/home/despoB/lesion/anat_preproc/lesion_mask_mni_stack.nii.gz'

In [4]:
mask_stack_img = nib.load(mask_stack_path)

In [5]:
mask_stack_data = mask_stack_img.get_data()

In [6]:
mask_stack_data.shape

(91, 109, 91, 64)

Get the inverse affine transform for the image, so we can go from image-space coordinates to voxel-data coordiantes.

In [7]:
mask_stack_img.affine

array([[  -2.,   -0.,    0.,   90.],
       [  -0.,    2.,   -0., -126.],
       [   0.,    0.,    2.,  -72.],
       [   0.,    0.,    0.,    1.]])

In [8]:
inverse_affine = np.linalg.inv(mask_stack_img.affine)

Confirm that our coordinate translation works by checking against a known relationship (from FSLView).

MNI coordinate (28, 36, 2) = Voxel coordinate (31, 81, 37)

In [9]:
nib.affines.apply_affine(inverse_affine, [28, 36, 2])

array([ 31.,  81.,  37.])

To be double sure, test our transformed coordinates on real data. The mask for patient 191 has a lesioned voxel at MNI coordinate (-36, 2, -2), but moving more than 2mm (1 voxel) in any direction puts you in empty space.

In [13]:
mask_path_191 = '/home/despoB/lesion/anat_preproc/191/191_mask_mni.nii.gz'

In [14]:
mask_img_191 = nib.load(mask_path_191)

In [46]:
mask_data_191 = mask_img_191.get_data()

There should be damage here..

In [63]:
nib.affines.apply_affine(inverse_affine, [-36, 2, -2])

array([ 63.,  64.,  35.])

In [64]:
mask_data_191[63, 64, 35]

1.0

But not in any of these places...

In [67]:
nib.affines.apply_affine(inverse_affine, [-40, 2, -2])

array([ 65.,  64.,  35.])

In [68]:
mask_data_191[65, 64, 35]

0.0

In [69]:
nib.affines.apply_affine(inverse_affine, [-36, -2, -2])

array([ 63.,  62.,  35.])

In [70]:
mask_data_191[63, 62, 35]

0.0

In [71]:
nib.affines.apply_affine(inverse_affine, [-36, 2, 2])

array([ 63.,  64.,  37.])

In [72]:
mask_data_191[63, 64, 37]

0.0

Looks like our coordinate conversion is working. These were lazy checks, but lazy is better than nothing.

Next, we need to map 3D voxel-data coordinates to 1D (flattened) coordinates.

In [10]:
%%timeit
np.where(mask_stack_data[37, 81, 31, :] == 1)

The slowest run took 14.48 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 4.47 µs per loop


In [11]:
mask_stack_data_flat = mask_stack_data.reshape(-1, 64).T

In [12]:
%%timeit 
np.where(mask_stack_data_flat[:,803425] == 0)

The slowest run took 16.99 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 4.2 µs per loop


In [15]:
mask_img_191.shape

(91, 109, 91)

In [16]:
mask_img_191.affine

array([[  -2.,   -0.,    0.,   90.],
       [  -0.,    2.,   -0., -126.],
       [   0.,    0.,    2.,  -72.],
       [   0.,    0.,    0.,    1.]])

In [19]:
mask_stack_data[...,0].shape

(91, 109, 91)

In [32]:
test_data = np.array([[[0, 1, 2], [3, 4, 5], [6, 7, 8]], [[9, 10, 11], [12, 13, 14], [15, 16, 17]]])

In [33]:
test_data.shape

(2, 3, 3)

In [34]:
test_data

array([[[ 0,  1,  2],
        [ 3,  4,  5],
        [ 6,  7,  8]],

       [[ 9, 10, 11],
        [12, 13, 14],
        [15, 16, 17]]])

In [41]:
test_data.ravel()[11]

11

In [39]:
test_data[1,0,2]

11

In [40]:
np.ravel_multi_index([1,0,2], (2,3,3))

11

In [20]:
import xarray as xr

In [21]:
mask_stack_data.T.shape

(64, 91, 109, 91)

In [73]:
xdata = xr.DataArray(mask_stack_data)

In [1]:
xdata

NameError: name 'xdata' is not defined

In [75]:
xdata.dim_3

<xarray.DataArray 'dim_3' (dim_3: 64)>
array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63])
Coordinates:
  * dim_3    (dim_3) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 ...

In [22]:
data = xr.DataArray(np.random.randn(2, 3), [('x', ['a', 'b']), ('y', [-2, 0, 2])])

In [23]:
data

<xarray.DataArray (x: 2, y: 3)>
array([[-1.65990267,  1.21584388,  1.55169129],
       [ 1.36510009,  0.01194809,  1.46804563]])
Coordinates:
  * x        (x) <U1 'a' 'b'
  * y        (y) int64 -2 0 2

In [25]:
data.loc['a']

<xarray.DataArray (y: 3)>
array([-1.65990267,  1.21584388,  1.55169129])
Coordinates:
    x        <U1 'a'
  * y        (y) int64 -2 0 2

In [26]:
[str(i) for i in range(64)]

['0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '40',
 '41',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '48',
 '49',
 '50',
 '51',
 '52',
 '53',
 '54',
 '55',
 '56',
 '57',
 '58',
 '59',
 '60',
 '61',
 '62',
 '63']

In [27]:
sdata = xr.DataArray(mask_stack_data.T, [('patient_id', [str(i) for i in range(64)])])

ValueError: coords is not dict-like, but it has 1 items, which does not match the 4 dimensions of the data